In [292]:
# !pip install implicit

In [293]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [294]:
# !wget url
# !unzip retail_train.csv.zip -d ../data > /dev/null

In [295]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [296]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [297]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [298]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [299]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [300]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = items_weights['item_id'].values
    weights = items_weights['weights'].values
    recs = np.random.choice(items, size=n, replace=False, p=weights)

    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [301]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights = items_weights[items_weights['sales_value']>1]
items_weights['weights']= np.log(items_weights['sales_value'] + 0.0000001)
items_weights.drop('sales_value', axis=1, inplace=True)
items_weights['weights'] = items_weights['weights'] / items_weights['weights'].sum()
# items_weights.head()

result['weighted_random_recommendation'] = result['user_id'].apply\
                                    (lambda x: weighted_random_recommendation(items_weights, n=5))                        

CPU times: user 2.83 s, sys: 26.4 ms, total: 2.85 s
Wall time: 2.86 s


In [302]:
result.head() 

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[7157315, 9836299, 1336524, 932761, 858829]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[14050831, 850004, 1013807, 5568860, 1011645]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[7443166, 1083358, 1134152, 15713433, 6904826]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1119051, 12810279, 15831353, 1029624, 9222012]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[778485, 938724, 10149851, 986006, 990789]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [303]:
# !cp predictions_basic.csv -d ../predictions > /dev/null

In [304]:
result = pd.read_csv('../predictions/predictions_basic.csv', converters={'actual': p, 
         'random_recommendation': eval, 'popular_recommendation': eval, 
         'itemitem': eval, 'cosine': eval, 'tfidf': eval, 'own_purchases': eval})
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [305]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    # bought_list = bought_list 
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / k
    
    return precision

In [310]:
# your_code

for col in result.iloc[:,2:]:
    print("Precision@5 для алгоритма", col, ": ","{0:}".format(result.apply
          (lambda row: precision_at_k(row[col], row['actual'], k=5), axis=1).mean()))

Precision@5 для алгоритма random_recommendation :  0.0005876591576885406
Precision@5 для алгоритма popular_recommendation :  0.15523996082272082
Precision@5 для алгоритма itemitem :  0.033594515181195064
Precision@5 для алгоритма cosine :  0.03525954946131262
Precision@5 для алгоритма tfidf :  0.03614103819784544
Precision@5 для алгоритма own_purchases :  0.17992164544563902


Лучший результат показали алгоритмы own_purchases и popular_recommendation 

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [308]:
# your_code